## Install

Fandango is installed as a python module and can be accessed through the command line. The easiest way to install is to run `pip install fandango-fuzzer`. After this, Fandango can be accessed through the terminal using `fandango` (or `python -m fandango`) in the same environment as where it was installed.

On Windows, if there are no pre-built wheels, Microsoft Visual Studio Build Tools and the Windows 10 SDK are required to build Fandango.

## Converting to the .fan specification

## Interaction using Python API


## Name Database (Demo)

You can create a database of name/age pairs by way of a grammar, stored in a .fan file as a set of rules dictating how the Fandango input should be generated. Note that <start> is the very first variable that the grammar should refer to.

Then, this command will generate 10 name, age pairs based on the grammar specified in persons.fan (an example .fan file).

fandango fuzz -f persons.fan -n 10

'-o FILENAME' redirects the output of the above command into a file FILENAME.
'-s DELIMITER' replaces the delimiter between outputs with DELIMITER (default is a newline).
'-d DIRECTORYNAME' stores each output in a directory named DIRECTORYNAME.

fandango fuzz -f persons.fan -n 10 -o persons.txt
fandango fuzz -f persons.fan -n 10 -o persons.txt -s ':'
fandango fuzz -f persons.fan -n 10 -d persons

The number of outputs can be limited via the numerical argument -n or -N, where -n refers to the number of solutions/outputs, and -N refers to the number of generated sets.

fandango fuzz -f persons.fan -n 10 # to limit the number of solutions
fandango fuzz -f persons.fan -N 10 # to limit the number of generations
